In [54]:
import nltk
import json
import spacy
from nltk.corpus import stopwords
from math import log
from collections import defaultdict, Counter
from string import punctuation
from nltk.stem.wordnet import WordNetLemmatizer
import gensim
import re
import csv

# Variables

In [46]:
OPEN_QUESTION_WORDS = ['what','who','whose','whom','where','when','why','how',
                       'which',"what's","who's","where's","how's"]
CLOSED_QUESTION_WORDS = ['is','are','am','was','were','do','does,','did','can',
                         'could','will','would','shall','should','have','has',
                         'had']

# Stop words
stop = set(stopwords.words('english'))

lmtz = WordNetLemmatizer()

with open('testing.json') as json_data:
    test = json.load(json_data)

with open('documents.json') as json_data:
    documents = json.load(json_data)

# Spacy toolkit
nlp = spacy.load('en_core_web_sm')

punc = set(punctuation)

In [47]:
def strip_punctuation(s):
    return ''.join(c for c in s if c not in punc)

In [48]:
def lemmatize(token):
    lemma = lmtz.lemmatize(token, 'v')
    if lemma == token:
        lemma = lmtz.lemmatize(token, 'n')
    return lemma

        
def extract_term_freqs(doc):
    tfs = {}
    for token in nltk.word_tokenize(doc):
        lemma = lemmatize(token.lower())
        if lemma not in stop and lemma.isalpha():
            tfs[lemma] = tfs.get(lemma, 0) + 1
    return tfs


def compute_doc_freqs(doc_term_freqs):
    dfs = Counter()
    for tfs in doc_term_freqs.values():
        for term in tfs.keys():
            dfs[term] += 1
    return dfs


def query_vsm(query, index, k=12):
    accumulator = Counter()
    for term in query:
        postings = index[term]
        for docid, weight in postings:
            accumulator[docid] += weight
    return accumulator.most_common(k)


# Find the question word
def get_qword(question):
    tokens = nltk.word_tokenize(question.lower())
    for token in tokens:
        if token in OPEN_QUESTION_WORDS:
            return token
    for token in tokens:
        if token in CLOSED_QUESTION_WORDS:
            return token
    return 'others'

In [49]:
# length of longest same sequences of keywords
def get_overlap(sent1, sent2):
    tokens1 = []
    tokens2 = []

    for token in nltk.word_tokenize(strip_punctuation(sent1.lower())):
        lemma = lemmatize(token)
        if lemma not in stop:
            tokens1.append(lemma)

    for token in nltk.word_tokenize(strip_punctuation(sent2.lower())):
        lemma = lemmatize(token)
        if lemma not in stop:
            tokens2.append(lemma)

    max = 0
    for i in range(len(tokens1)):
        for j in range(len(tokens2)):

            if tokens1[i] == tokens2[j]:
                length = 1

                ii = i + 1
                jj = j + 1
                while ii < len(tokens1) and jj < len(tokens2) and \
                        tokens1[ii] == tokens2[jj]:
                    ii += 1
                    jj += 1
                    length += 1

                if length > max:
                    max = length

    return max

# Write to test file

In [65]:
csvFile = open("high.csv", "w")
writer = csv.writer(csvFile)
header = ['id','answer']
writer.writerow(header)


case_count = 0
# test = [test[17]]
for test_case in test:
    question = test_case['question']
    docid = test_case['docid']

    # Convert doc into one string, then tokenize sentences
    corpus = ''
    for para in documents[docid]['text']:
        corpus += para + ' '

    # sentence as a document
    raw_docs = nltk.sent_tokenize(corpus)
    
#     fn_docs = raw_docs
#     bm25 = BM25(fn_docs, delimiter=' ')
#     Query = question
#     Query = Query.split()
#     scores = bm25.BM25Score(Query)
#     tfidf = bm25.TFIDF()
#     print bm25.Items()
    
#     for i, tfidfscore in enumerate(tfidf):
#         print i, tfidfscore
    # TFIDF
    doc_term_freqs = {}
    for (id, raw_doc) in enumerate(raw_docs):
        term_freqs = extract_term_freqs(raw_doc)
        doc_term_freqs[id] = term_freqs
    M = len(doc_term_freqs)

    doc_freqs = compute_doc_freqs(doc_term_freqs)

    vsm_inverted_index = defaultdict(list)
    for docid, term_freqs in doc_term_freqs.items():
        N = sum(term_freqs.values())
        length = 0

        # find tf*idf values and accumulate sum of squares
        tfidf_values = []
        for term, count in term_freqs.items():
            tfidf = float(count) / N * log(M / float(doc_freqs[term]))
            tfidf_values.append((term, tfidf))
            length += tfidf ** 2

        # normalise documents by length and insert into index
        length = length ** 0.5
        for term, tfidf in tfidf_values:
            # inversion of the indexing, term -> (doc_id, score)
            vsm_inverted_index[term].append([docid, tfidf / length])

    for term, docids in vsm_inverted_index.items():
        docids.sort()

    terms = extract_term_freqs(question)
    results = query_vsm(terms, vsm_inverted_index)


    # Step 2
    # Analyse question type
    qword = get_qword(question)

    # the word after question word, such as 'what value', 'which gender'
    next_token = ''

    qtype = ''

    # dependency parsing
    dep = ''

    # head word
    head = ''

    # head dependency
    head_dep = ''

    # subject, root, object
    nsubj = ''
    ROOT = ''
    dobj = ''

    # yes or no questions have two options
    closed_q_choices = ('', '')

    doc = nlp(question)

    tokens = nltk.word_tokenize(question.lower())

    # get next word
    if qword in tokens:
        if tokens.index(qword) < len(tokens) - 1:
            next_token = tokens[tokens.index(qword) + 1]

    # get structure of sentence
    for token in doc:
        if 'nsubj' in token.dep_:
            nsubj = lemmatize(strip_punctuation(token.text))
        if token.dep_ == 'ROOT':
            ROOT = lemmatize(strip_punctuation(token.text))
        if 'dobj' in token.dep_:
            dobj = lemmatize(strip_punctuation(token.text))

    # for noun (phrase) questions, get answer dependency
    for chunk in doc.noun_chunks:
        if qword in chunk.text:
            dep = chunk.root.dep_
            head = lemmatize(strip_punctuation(chunk.root.head.text))
            head_dep = chunk.root.head.dep_

    # determine answer type
    if 'stand for' in question or 'abbreviat' in question:
        qtype = 'abrv'

    elif qword in ['who',"who's",'whom','whose']:
        qtype = 'who'

    elif qword == 'when':
        qtype = 'when'

    elif qword in ['where',"where's"]:
        qtype = 'where'

    elif qword in ['how',"how's"]:
        if next_token == 'much':
            qtype = 'MONEY'
        elif next_token == 'many':
            qtype = 'CARDINAL'
        elif next_token == 'long':
            qtype = 'DATE'
        elif next_token in ['far','big','wide','deep','tall','high','fast','heavy']:
            qtype = 'QUANTITY'
        elif next_token in ['old','young']:
            qtype = 'DATE'
        elif next_token in ['does','did','do','have','has','had','should',
                              'can','could','will','would','must']:
            if dobj != '':
                qtype = 'adj'
            else:
                qtype = 'verb'

    elif qword in ['what', "what's", 'which']:

        if 'year'in tokens or \
                'day' in tokens or \
                'month' in tokens or \
                'era' in tokens or \
                'age' in tokens or \
                'century' in tokens or \
                'week' in tokens or \
                'period' in tokens or \
                'dynasty' in tokens:
            qtype = 'DATE'

        elif 'company' in tokens or \
                'organization' in tokens or \
                'organisation' in tokens or \
                'corporation' in tokens or \
                'institution' in tokens or \
                'university' in tokens or \
                'corporation' in tokens or \
                'association' in tokens or \
                'union' in tokens or \
                'agency' in tokens:
            qtype = 'ORG'

        elif 'city' in tokens or \
                'country' in tokens or \
                'state' in tokens or \
                'province' in tokens or \
                'county' in tokens:
            qtype = 'GPE'

        elif 'place' in tokens or \
                'river' in tokens or \
                'mountain' in tokens or \
                'ocean' in tokens or \
                'region' in tokens or \
                'area' in tokens or \
                'sea' in tokens or \
                'lake' in tokens or \
                'continent' in tokens or \
                'location' in tokens or \
                'forest' in tokens or \
                'jungle' in tokens:
            qtype = 'LOC'

        elif 'nationality' in tokens:
            qtype = 'NORP'

        elif 'building' in tokens or \
            'airport' in tokens or \
            'highway' in tokens or \
            'bridge' in tokens or \
            'harbour' in tokens or \
            'harbor' in tokens or \
            'port' in tokens or \
            'dam' in tokens:
            qtype = 'FACILITY'

        elif 'hurricane' in tokens or \
            'battle' in tokens or \
            'war' in tokens:
            qtype = 'EVENT'

        elif 'book' in tokens or \
            'novel' in tokens or \
            'song' in tokens or \
            'music' in tokens or \
            'painting' in tokens:
            qtype = 'WORK_OF_ART'

        elif 'language' in tokens or \
                'speak' in tokens:
            qtype = 'LANGUAGE'

        elif 'percentage' in tokens or 'percent' in tokens:
            qtype = 'PERCENT'

        elif 'value' in tokens or \
                'distance' in tokens or \
                'size' in tokens or \
                'length' in tokens or \
                'depth' in tokens or \
                'height' in tokens or \
                'density' in tokens or \
                'speed' in tokens or \
                'weight' in tokens or \
                'area' in tokens or \
                'temperature' in tokens or \
                'volume' in tokens:
            qtype = 'QUANTITY'

        elif 'number' in tokens:
            qtype = 'CARDINAL'

        elif 'price' in tokens:
            qtype = 'MONEY'

        else:
            # what...do type question
            tokens.remove(next_token)
            if 'do' in tokens:
                qtype = 'verb'
            else:
                qtype = 'noun'

    elif qword == 'why':
        qtype = 'why'

    elif qword in CLOSED_QUESTION_WORDS:
        qtype = 'closed'

        # answer is one of the 'or' options in the question
        if 'or' in tokens:
            index = tokens.index('or')
            prev1 = tokens[index - 1]
            next1 = tokens[index + 1]
            tag_tokens = nltk.pos_tag(tokens)

            tag = tag_tokens[index - 1][1]

            # if answer is a noun
            if tag in ['NN', 'NNP', 'NNS', 'NNPS']:
                for chunk in doc.noun_chunks:
                    if prev1 in chunk.text:
                        first = chunk.text
                    if next1 in chunk.text:
                        second = chunk.text
                closed_q_choices = (first, second)
            else:
                closed_q_choices = (prev1, next1)
        else:
            qtype = 'others'

    # re-rank the 20 sentences
    scores = {}
    for id, _ in results:
        sent = raw_docs[id]
        doc = nlp(sent)

        score = get_overlap(sent, question)

        if qtype == 'who':
            for ent in doc.ents:
                if ent.label_ == 'PERSON':
                    score += 1

        elif qtype == 'when':
            for ent in doc.ents:
                if ent.label_ == 'TIME' or ent.label_ == "DATE":
                    score += 1

        elif qtype == 'where':
            for ent in doc.ents:
                if ent.label_ == 'GPE' or ent.label_ == "LOC":
                    score += 1

        elif qtype in ['LANGUAGE','WORK_OF_ART','EVENT','NORP','FACILITY',
                       'GPE','DATE','TIME','PERCENT','QUANTITY','CARDINAL',
                     'MONEY','PERSON','ORG','LOC']:
            for ent in doc.ents:
                if ent.label_ == qtype:
                    score += 1

        elif qtype == 'adj':
            for token in doc:
                if 'advmod' in token.dep_ or 'acomp' in token.dep_:
                    score += 1

        elif qtype == 'verb':
            for token in doc:
                if token.dep_ == 'ROOT':
                    score += 1

        elif qtype == 'closed':
            first = closed_q_choices[0]
            second = closed_q_choices[1]

            score += (first in sent) + (second in sent)

        elif qtype == 'why':
            if 'reason' in sent or 'because' in sent or 'due to' in sent or 'since' in sent or 'for' in sent:
                score += 1

        scores[id] = score

    rank = {}
    for id, sim in results:
        max_score = scores[max(scores, key=scores.get)]
        if max_score != 0:
            rank[id] = sim * 0.5 + (scores[id] / max_score * 0.5)
        else:
            rank[id] = sim
    
    # sentence with highest rank
    index = max(rank, key=rank.get)
    sent = raw_docs[index]
    doc = nlp(sent)

    # find sentence structure
    sent_nsubj = ''
    sent_ROOT = ''
    sent_dobj = ''
    for token in doc:
        if 'nsubj' in token.dep_:
            sent_nsubj = lemmatize(strip_punctuation(token.text))
        if token.dep_ == 'ROOT':
            sent_ROOT = lemmatize(strip_punctuation(token.text))
        if 'dobj' in token.dep_:
            sent_dobj = lemmatize(strip_punctuation(token.text))
            
    # find answer with highest score
    max_score = -1
    answer = ''
    
    if qtype == 'who':
        for np in doc.noun_chunks:
            score = 0
            
            if np in doc.ents:
                for ent in doc.ents:
                    if np.text in ent.text and ent.label_ == 'PERSON':
                            score += 3

            # find NP dependency
            np_dep = np.root.dep_
            np_head = lemmatize(strip_punctuation(np.root.head.text))
            np_head_dep = np.root.head.dep_

            if np_dep == dep:
                score += 1
            if np_head == head:
                score += 1
            if np_head_dep == head_dep:
                score += 1

            if np.text not in question:
                score += 1

            if strip_punctuation(np.text).strip().lower() not in stop:
                score += 1

            if np.text.lower() == 'it':
                score = -1
                
            if score > max_score:
                max_score = score
                answer = np.text

    elif qtype == 'when':
        for ent in doc.ents:
            score = 0
            
            if ent.label_ == 'TIME' or ent.label_ == "DATE":
                score += 3
                
            if ent.text not in question:
                score += 1
            
            if score > max_score:
                max_score = score
                answer = ent.text

    elif qtype == 'where':
        for ent in doc.ents:
            score = 0
            
            if ent.label_ == 'GPE' or ent.label_ == "LOC":
                score += 3

            if ent.text not in question:
                score += 1
                
            if score > max_score:
                max_score = score
                answer = ent.text
            
    elif qtype in ['LANGUAGE', 'WORK_OF_ART', 'EVENT', 'NORP', 'FACILITY',
                   'GPE', 'DATE', 'TIME', 'PERCENT', 'QUANTITY', 'CARDINAL',
                   'MONEY', 'PERSON', 'ORG', 'LOC']:
        for ent in doc.ents:
            score = 0
            
            if ent.label_ == qtype:
                score += 3
            
            if ent.text not in question:
                score += 1
                
            if score > max_score:
                max_score = score
                answer = ent.text

    elif qtype == 'abrv':
        abrv = ''
        qdoc = nlp(question)
        for token in qdoc:
            text = token.text
            if len(text) >= 2 and text.isupper() and text.isalpha():
                abrv = text.lower()

        if abrv == '' and 'stand for' in question:
            tokens = question.lower().split(' ')
            abrv = tokens[tokens.index('stand')-1]

        if abrv != '':
            tokens = nltk.word_tokenize(sent)
            for (i, token) in enumerate(tokens):
                if token[0].isupper():
                    k = 1
                    phrase = token.lower()
                    initials = phrase[0]

                    while i+k < len(tokens) and tokens[i+k][0].isupper():
                        phrase = phrase + ' ' + tokens[i+k].lower()
                        initials += tokens[i+k][0].lower()
                        k += 1

                    phrase = phrase.strip()
                    if initials == abrv:
                        answer = phrase

        else:
            tokens = nltk.word_tokenize(question)
            for (i, token) in enumerate(tokens):
                if token[0].isupper():
                    k = 1
                    initials = token[0].lower()

                    while i + k < len(tokens) and tokens[i + k][0].isupper():
                        initials += tokens[i + k][0].lower()
                        k += 1

                    if len(initials) >= 2:
                        answer = initials

    elif qtype == 'adj':
        for token in doc:
            score = 0
            
            if 'advmod' in token.dep_ or 'acomp' in token.dep_:
                score += 3

            token_dep = token.dep_
            token_head = lemmatize(strip_punctuation(token.head.text))
            token_head_dep = token.head.dep_

            if token_dep == dep:
                score += 1
            if token_head == head:
                score += 1
            if token_head_dep == head_dep:
                score += 1

            if token.text not in question:
                score += 1

            if strip_punctuation(token.text).strip().lower() not in stop:
                score += 1

            if token.text.lower() == 'it':
                score = -1

            if score > max_score:
                max_score = score
                answer = token.text

    elif qtype == 'verb':
        for token in doc:
            score = 0

            if token.dep_ == 'ROOT':
                score += 1

            if lemmatize(strip_punctuation(token.text)) not in \
                    [lemmatize(strip_punctuation(s)) for s in nltk.word_tokenize(question)]:
                score += 1

            if strip_punctuation(token.text).strip().lower() not in stop:
                score += 1

            if score > max_score:
                max_score = score
                answer = token.text

    elif qtype == 'closed':
        first = closed_q_choices[0]
        second = closed_q_choices[1]

        # whether each option appears (and is negates)
        appear1 = False
        appear2 = False
        negate1 = False
        negate2 = False
        neg_count = 0
        tokens = nltk.word_tokenize(raw_docs[id])

        for (index, token) in enumerate(tokens):
            if token == 'not' or "n't" in token:
                neg_count += 1

                if index+1 < len(tokens):
                    if tokens[index+1] == first:
                        negate1 = True
                    if tokens[index+1] == second:
                        negate2 = True

            if token == first:
                appear1 = True
            if token == second:
                appear2 = True

        possible_answer = ''
        if appear1 and not appear2:
            if neg_count % 2 == 1:
                possible_answer = second
            else:
                possible_answer = first

        elif appear2 and not appear1:
            if neg_count % 2 == 0:
                possible_answer = second
            else:
                possible_answer = first

        elif appear1 and appear2:
            if negate1 and not negate2:
                possible_answer = second
            elif negate2 and not negate1:
                possible_answer = first
            else:
                possible_answer = second

        if possible_answer != '':
            score += 5
            if score > max_score:
                max_score = score
                answer = possible_answer

    elif qtype == 'why':

        possible_answer = ''
        score = 0

        if 'reason' in sent or 'because' in sent or 'due to' in sent or 'since' in sent or 'for' in sent:

            if 'because of' in sent:
                score += 3
                index = sent.index('because of')
                substr = sent[index+11:]
                span = nlp(substr)
                for chunk in span.noun_chunks:
                    possible_answer = chunk.text
                    break

            elif 'because' in sent:
                score += 3
                index = sent.index('because')
                substr = sent[index + 8:]
                possible_answer = substr

            elif 'due to' in sent:
                score += 3
                index = sent.index('due to')
                substr = sent[index+7:]
                span = nlp(substr)
                for chunk in span.noun_chunks:
                    possible_answer = chunk.text
                    break
                if possible_answer == '':
                    possible_answer = substr

            elif 'reason' in sent:
                score += 2
                index = sent.index('reason')
                substr = sent[index+7:]
                span = nlp(substr)
                for chunk in span.noun_chunks:
                    possible_answer = chunk.text
                    break
                if possible_answer == '':
                    index = substr.find('is')
                    if index != -1:
                        possible_answer = substr[index+3]
                    else:
                        index = substr.find('was')
                        if index != -1:
                            possible_answer = substr[index+4]
                        else:
                            possible_answer = sent[sent.index('reason'):]

            elif 'for' in sent:
                score += 1
                index = sent.index('for')
                substr = sent[index + 4:]
                span = nlp(substr)
                for chunk in span.noun_chunks:
                    possible_answer = chunk.text
                    break
                if possible_answer == '':
                    possible_answer = substr

            elif 'since' in sent:
                score += 1
                index = sent.index('since')
                substr = sent[index + 6:]
                possible_answer = substr

            if possible_answer != '' and score > max_score:
                answer = possible_answer
                max_score = score

    # if answer not found, find noun phrases
    if answer == '':
        for np in doc.noun_chunks:
            score = 0

            np_dep = np.root.dep_
            np_head = lemmatize(strip_punctuation(np.root.head.text))
            np_head_dep = np.root.head.dep_

            if np_dep == dep:
                score += 1
            if np_head == head:
                score += 1
            if np_head_dep == head_dep:
                score += 1

            if np.text not in question:
                score += 1

            if strip_punctuation(np.text).strip().lower() not in stop:
                score += 1

            if np.text.lower() == 'it':
                score = 0

            if score > max_score:
                max_score = score
                answer = np.text
                
                
    a = nltk.word_tokenize(answer)
    try:
        if a[0].lower() == 'the' or a[0].lower() == 'a' or a[0].lower() == 'an' or a[0].lower() == 'this':
            answer = ''
            del a[0]
            for word in a:
                answer += word + ' '
    except:
        answer = answer
        
    answer = answer.strip().lower()
    result = []
    result.append(case_count)
    result.append(answer)
    writer.writerow(result)
    print(case_count,' ',answer)
    case_count += 1
    
csvFile.close()

0   combination
1   addition
2   browser 's layout engine
3   its opera-mini version
4   late 2004
5   windows
6   1995
7   most browsers
8   marc andreessen
9   first web browser
10   competition
11   dominance
12   internet relay chat
13   january
14   every major web browser
15   january 2003
16   january 2009
17   file transfer protocol
18   google
19   case
20   rich user interfaces
21   2002
22   august 2011
23   all major web browsers
24   chrome's user-base
25   development
26   december 2011
27   january 2003
28   netscape
29   apple's safari
30   rapid development
31   prefix
32   mozilla foundation
33   private networks
34   mac
35   comparison
36   1994
37   user interface
38   addition
39   major browsers
40   information resources
41   live bookmarks
42   more traditional feed reader
43   bookmarks
44   year
45   prefix
46   browser software
47   file transfer protocol
48   microsoft corp
49   mobile safari
50   most web browsers
51   windows
52   user 's default e-mail a

459   city
460   additional police stations
461   local train services
462   two
463   uk
464   solent
465   p&o cruises
466   27 june 1640
467   college
468   840
469   centuries
470   1233
471   university hospital southampton nhs foundation trust
472   biggest operator
473   4.2
474   1959
475   354
476   government figures
477   m27
478   talking heads
479   hanover buildings
480   city
481   december 2007
482   12th century
483   traffic congestion
484   southampton
485   2004
486   1233
487   built-up area
488   three fire stations
489   southampton docks
490   24
491   western docks
492   southampton's largest retail centre
493   1920s
494   1976
495   clausentum
496   two
497   plans
498   southampton
499   port
500   just over a quarter
501   main station
502   20–24
503   council estates
504   south west trains
505   16.2 percent
506   king henry's departure
507   route
508   trust
509   town
510   16.2 percent
511   two large live music venues
512   three
513   university of

905   neptune's more varied weather
906   30.1
907   pluto
908   apparent backward motion
909   current models
910   1997
911   planetary science
912   400 m/s
913   2004
914   images
915   1017
916   other candidates
917   neptune's more varied weather
918   reaction
919   hobbes
920   modern term `` culture
921   non-material culture
922   citation
923   19th-century
924   terror management theory
925   29
926   united states
927   germany
928   end of the last ice age
929   other 19th-century critics
930   contrasted
931   terror management theory
932   anthropologists
933   1970s
934   around 50,000 years ago
935   united states
936   diffusion
937   prussian linguist
938   1950s and 1960s
939   stuart hall
940   élite ideal
941   present legislation
942   jefferson's metaphor
943   1971
944   nonconformists
945   all areas
946   pervasive secularism
947   that wall
948   may 3, 2006
949   still other scholars
950   as early as the mid-17th century
951   religious freedom
952   eng

1312   15,000
1313   four
1314   russians
1315   both sides
1316   william howard russell
1317   time
1318   1857
1319   25 september
1320   constantinople
1321   war
1322   february
1323   morning
1324   piedmont
1325   nicholas
1326   late september
1327   year
1328   three
1329   bulgaria
1330   alexander ii
1331   french
1332   cronstadt
1333   second
1334   walachia
1335   minor naval skirmishes
1336   ukraine
1337   19–31
1338   black sea
1339   31 december
1340   cronstadt
1341   george hamilton-gordon
1342   omar pasha
1343   32–40
1344   its alliance
1345   local commanders
1346   russians
1347   william howard russell
1348   constantinople
1349   omar pasha
1350   omar pasha
1351   parliament
1352   constantinople
1353   åland islands
1354   alexander ii
1355   nicholas
1356   cardigan
1357   treaty of paris
1358   austria
1359   ridge
1360   october 1853
1361   french
1362   united kingdom
1363   centuries-old
1364   public opinion
1365   second counterattack
1366   winter o

1774   regions
1775   united kingdom
1776   sunlight exposure
1777   unexpected adverse effect
1778   winston churchill
1779   people
1780   proposal
1781   1907
1782   1995
1783   microsoft windows
1784   negative dst
1785   britain
1786   standard time
1787   autumn and two hours
1788   merriam-webster's
1789   two hours
1790   dst
1791   correlation
1792   (cet
1793   all years
1794   fixed work schedules
1795   example
1796   dst
1797   november
1798   several studies
1799   1970s
1800   older form
1801   march 2011
1802   2007
1803   us
1804   winter
1805   north america
1806   1895
1807   ntfs
1808   is
1809   northern summer
1810   some farmers' groups
1811   some countries
1812   dst inherits
1813   dst
1814   filesystem
1815   2007
1816   merriam-webster
1817   some farmers' groups
1818   st
1819   people
1820   time zone differences
1821   early goal
1822   one reason
1823   daylight
1824   common agreement
1825   kingsford charcoal
1826   1999 study
1827   many enactments
18

2182   only seven
2183   whites
2184   categories
2185   biological race concept
2186   population
2187   existence
2188   scientific classification
2189   subspecies
2190   sex
2191   rigid descent rule
2192   forensic physical anthropologist
2193   1949
2194   skeletal remain
2195   at least two
2196   sense
2197   falsify
2198   recent work
2199   no generally accepted concept
2200   greatest number
2201   many
2202   survey
2203   races
2204   about 30%
2205   clustering
2206   today
2207   20th century
2208   2000
2209   brazilian child
2210   racism
2211   suggest
2212   populations
2213   cladistics
2214   phylogenetic analysis
2215   initial hypotheses
2216   east asians
2217   populations
2218   practice
2219   significant number
2220   700.000
2221   clade
2222   diagnosis
2223   concept
2224   eduardo bonilla-silva
2225   physical anthropologists
2226   adversely
2227   mass incarceration
2228   many thousands
2229   france
2230   y chromosomes
2231   about 5%
2232   roughly

2582   gabriel
2583   citation
2584   case
2585   complaint
2586   articles
2587   many treaties
2588   other treaties
2589   2008
2590   dates
2591   ruled
2592   treaties
2593   add
2594   articles
2595   states
2596   reservations
2597   supreme court
2598   passenger elevator
2599   40-50%
2600   1853
2601   weight
2602   counterweights
2603   opens
2604   berlin
2605   werner von siemens
2606   elevator cab
2607   germany
2608   low-volume hours
2609   double deck elevators
2610   make
2611   citation
2612   israeli
2613   hydraulic crane
2614   four
2615   relay controller
2616   city transport
2617   governor device
2618   passenger cabs
2619   unique design characteristics
2620   residential
2621   each end
2622   500 ft/min
2623   may
2624   approximately 300,000
2625   passenger elevators
2626   less expensive installations
2627   1
2628   gearless traction machines
2629   these steam driven devices
2630   cab interiors
2631   elevator cars
2632   dumbwaiters
2633   freight e

2988   2011
2989   its software business
2990   2.06 percent
2991   virbhadra singh
2992   state-owned television broadcaster
2993   governor
2994   himachal
2995   sikhs
2996   kangra miniature paintings
2997   state
2998   himachal
2999   state
3000   himachal pradesh
3001   economy
3002   10th century
3003   hill stations
3004   public radio station
3005   1 november 1956
3006   himachal
3007   handicrafts
3008   virbhadra singh
3009   21st
3010   anglo-gorkha war
3011   superintendent
3012   pit-loom
3013   virbhadra singh
3014   buddhism and sikhism
3015   himachal pradesh
3016   wool
3017   year 1768
3018   hima
3019   hydro electric resources
3020   dial-up access
3021   mahmud ghaznavi
3022   citation
3023   citation
3024   technological advancements
3025   30 december 2007
3026   first
3027   nepal
3028   h.p.
3029   private fm stations
3030   18 december 1970
3031   state
3032   1.16%
3033   radio
3034   2006
3035   handicrafts
3036   1948
3037   muslims
3038   first five-yea

3395   romanian
3396   ming dynasty
3397   latin
3398   , a more sociolinguistic notion
3399   multiple standard dialects
3400   latin
3401   latin
3402   italians
3403   swedish
3404   dialect
3405   serbia
3406   1996
3407   tok pisin
3408   public environment
3409   25.1, 1945
3410   low german varieties
3411   north african
3412   first
3413   spanish
3414   rest
3415   only 2.5%
3416   english
3417   tunisia
3418   ion bărbuţă
3419   risorgimento
3420   heinz kloss
3421   italian
3422   south
3423   world war i
3424   modern nationalism
3425   opposite example
3426   arabic
3427   arabic
3428   yiddish speaker
3429   northern germany
3430   terms `` language
3431   them
3432   example
3433   northern italy's
3434   hong kong
3435   standardized dialect
3436   various terms
3437   italy
3438   german empire
3439   german language
3440   that interpretation
3441   secondary sense
3442   dialect
3443   english
3444   dialect
3445   standard dialect
3446   german dialectology
3447   i

#  准确率测试

In [19]:
with open('training.json') as json_data:
    train = json.load(json_data)
    
case_count = 0

for train_case in train:
    question = train_case['question']
    docid = train_case['docid']
    correct_answer = train_case['text']

    # Convert doc into one string, then tokenize sentences
    corpus = ''
    for para in documents[docid]['text']:
        corpus += para + ' '

    # sentence as a document
    raw_docs = nltk.sent_tokenize(corpus)

    # TFIDF
    doc_term_freqs = {}
    for (id, raw_doc) in enumerate(raw_docs):
        term_freqs = extract_term_freqs(raw_doc)
        doc_term_freqs[id] = term_freqs
    M = len(doc_term_freqs)

    doc_freqs = compute_doc_freqs(doc_term_freqs)

    vsm_inverted_index = defaultdict(list)
    for docid, term_freqs in doc_term_freqs.items():
        N = sum(term_freqs.values())
        length = 0

        # find tf*idf values and accumulate sum of squares
        tfidf_values = []
        for term, count in term_freqs.items():
            tfidf = float(count) / N * log(M / float(doc_freqs[term]))
            tfidf_values.append((term, tfidf))
            length += tfidf ** 2

        # normalise documents by length and insert into index
        length = length ** 0.5
        for term, tfidf in tfidf_values:
            # inversion of the indexing, term -> (doc_id, score)
            vsm_inverted_index[term].append([docid, tfidf / length])

    for term, docids in vsm_inverted_index.items():
        docids.sort()

    terms = extract_term_freqs(question)
    results = query_vsm(terms, vsm_inverted_index)


    # Step 2
    # Analyse question type
    qword = get_qword(question)

    # the word after question word, such as 'what value', 'which gender'
    next_token = ''

    qtype = ''

    # dependency parsing
    dep = ''

    # head word
    head = ''

    # head dependency
    head_dep = ''

    # subject, root, object
    nsubj = ''
    ROOT = ''
    dobj = ''

    # yes or no questions have two options
    closed_q_choices = ('', '')

    doc = nlp(question)

    tokens = nltk.word_tokenize(question.lower())

    # get next word
    if qword in tokens:
        if tokens.index(qword) < len(tokens) - 1:
            next_token = tokens[tokens.index(qword) + 1]

    # get structure of sentence
    for token in doc:
        if 'nsubj' in token.dep_:
            nsubj = lemmatize(strip_punctuation(token.text))
        if token.dep_ == 'ROOT':
            ROOT = lemmatize(strip_punctuation(token.text))
        if 'dobj' in token.dep_:
            dobj = lemmatize(strip_punctuation(token.text))

    # for noun (phrase) questions, get answer dependency
    for chunk in doc.noun_chunks:
        if qword in chunk.text:
            dep = chunk.root.dep_
            head = lemmatize(strip_punctuation(chunk.root.head.text))
            head_dep = chunk.root.head.dep_

    # determine answer type
    if 'stand for' in question or 'abbreviat' in question:
        qtype = 'abrv'

    elif qword in ['who',"who's",'whom','whose']:
        qtype = 'who'

    elif qword == 'when':
        qtype = 'when'

    elif qword in ['where',"where's"]:
        qtype = 'where'

    elif qword in ['how',"how's"]:
        if next_token == 'much':
            qtype = 'MONEY'
        elif next_token == 'many':
            qtype = 'CARDINAL'
        elif next_token == 'long':
            qtype = 'DATE'
        elif next_token in ['far','big','wide','deep','tall','high','fast','heavy']:
            qtype = 'QUANTITY'
        elif next_token in ['old','young']:
            qtype = 'DATE'
        elif next_token in ['does','did','do','have','has','had','should',
                              'can','could','will','would','must']:
            if dobj != '':
                qtype = 'adj'
            else:
                qtype = 'verb'

    elif qword in ['what', "what's", 'which']:

        if 'year'in tokens or \
                'day' in tokens or \
                'month' in tokens or \
                'era' in tokens or \
                'age' in tokens or \
                'century' in tokens or \
                'week' in tokens or \
                'period' in tokens or \
                'dynasty' in tokens:
            qtype = 'DATE'

        elif 'company' in tokens or \
                'organization' in tokens or \
                'organisation' in tokens or \
                'corporation' in tokens or \
                'institution' in tokens or \
                'university' in tokens or \
                'corporation' in tokens or \
                'association' in tokens or \
                'union' in tokens or \
                'agency' in tokens:
            qtype = 'ORG'

        elif 'city' in tokens or \
                'country' in tokens or \
                'state' in tokens or \
                'province' in tokens or \
                'county' in tokens:
            qtype = 'GPE'

        elif 'place' in tokens or \
                'river' in tokens or \
                'mountain' in tokens or \
                'ocean' in tokens or \
                'region' in tokens or \
                'area' in tokens or \
                'sea' in tokens or \
                'lake' in tokens or \
                'continent' in tokens or \
                'location' in tokens or \
                'forest' in tokens or \
                'jungle' in tokens:
            qtype = 'LOC'

        elif 'nationality' in tokens:
            qtype = 'NORP'

        elif 'building' in tokens or \
            'airport' in tokens or \
            'highway' in tokens or \
            'bridge' in tokens or \
            'harbour' in tokens or \
            'harbor' in tokens or \
            'port' in tokens or \
            'dam' in tokens:
            qtype = 'FACILITY'

        elif 'hurricane' in tokens or \
            'battle' in tokens or \
            'war' in tokens:
            qtype = 'EVENT'

        elif 'book' in tokens or \
            'novel' in tokens or \
            'song' in tokens or \
            'music' in tokens or \
            'painting' in tokens:
            qtype = 'WORK_OF_ART'

        elif 'language' in tokens or \
                'speak' in tokens:
            qtype = 'LANGUAGE'

        elif 'percentage' in tokens or 'percent' in tokens:
            qtype = 'PERCENT'

        elif 'value' in tokens or \
                'distance' in tokens or \
                'size' in tokens or \
                'length' in tokens or \
                'depth' in tokens or \
                'height' in tokens or \
                'density' in tokens or \
                'speed' in tokens or \
                'weight' in tokens or \
                'area' in tokens or \
                'temperature' in tokens or \
                'volume' in tokens:
            qtype = 'QUANTITY'

        elif 'number' in tokens:
            qtype = 'CARDINAL'

        elif 'price' in tokens:
            qtype = 'MONEY'

        else:
            # what...do type question
            tokens.remove(next_token)
            if 'do' in tokens:
                qtype = 'verb'
            else:
                qtype = 'noun'

    elif qword == 'why':
        qtype = 'why'

    elif qword in CLOSED_QUESTION_WORDS:
        qtype = 'closed'

        # answer is one of the 'or' options in the question
        if 'or' in tokens:
            index = tokens.index('or')
            prev1 = tokens[index - 1]
            next1 = tokens[index + 1]
            tag_tokens = nltk.pos_tag(tokens)

            tag = tag_tokens[index - 1][1]

            # if answer is a noun
            if tag in ['NN', 'NNP', 'NNS', 'NNPS']:
                for chunk in doc.noun_chunks:
                    if prev1 in chunk.text:
                        first = chunk.text
                    if next1 in chunk.text:
                        second = chunk.text
                closed_q_choices = (first, second)
            else:
                closed_q_choices = (prev1, next1)
        else:
            qtype = 'others'

    # re-rank the 20 sentences
    scores = {}
    for id, _ in results:
        sent = raw_docs[id]
        doc = nlp(sent)

        score = get_overlap(sent, question)

        if qtype == 'who':
            for ent in doc.ents:
                if ent.label_ == 'PERSON':
                    score += 1

        elif qtype == 'when':
            for ent in doc.ents:
                if ent.label_ == 'TIME' or ent.label_ == "DATE":
                    score += 1

        elif qtype == 'where':
            for ent in doc.ents:
                if ent.label_ == 'GPE' or ent.label_ == "LOC":
                    score += 1

        elif qtype in ['LANGUAGE','WORK_OF_ART','EVENT','NORP','FACILITY',
                       'GPE','DATE','TIME','PERCENT','QUANTITY','CARDINAL',
                     'MONEY','PERSON','ORG','LOC']:
            for ent in doc.ents:
                if ent.label_ == qtype:
                    score += 1

        elif qtype == 'adj':
            for token in doc:
                if 'advmod' in token.dep_ or 'acomp' in token.dep_:
                    score += 1

        elif qtype == 'verb':
            for token in doc:
                if token.dep_ == 'ROOT':
                    score += 1

        elif qtype == 'closed':
            first = closed_q_choices[0]
            second = closed_q_choices[1]

            score += (first in sent) + (second in sent)

        elif qtype == 'why':
            if 'reason' in sent or 'because' in sent or 'due to' in sent or 'since' in sent or 'for' in sent:
                score += 1

        scores[id] = score

    rank = {}
    for id, sim in results:
        max_score = scores[max(scores, key=scores.get)]
        if max_score != 0:
            rank[id] = sim * 0.5 + (scores[id] / max_score * 0.5)
        else:
            rank[id] = sim
    
    # sentence with highest rank
    index = max(rank, key=rank.get,default=0)
    sent = raw_docs[index]
    doc = nlp(sent)

    
    # find sentence structure
    sent_nsubj = ''
    sent_ROOT = ''
    sent_dobj = ''
    for token in doc:
        if 'nsubj' in token.dep_:
            sent_nsubj = lemmatize(strip_punctuation(token.text))
        if token.dep_ == 'ROOT':
            sent_ROOT = lemmatize(strip_punctuation(token.text))
        if 'dobj' in token.dep_:
            sent_dobj = lemmatize(strip_punctuation(token.text))
            
    # find answer with highest score
    max_score = -1
    answer = ''
    
    if qtype == 'who':
        for np in doc.noun_chunks:
            score = 0
            
            if np in doc.ents:
                for ent in doc.ents:
                    if np.text in ent.text and ent.label_ == 'PERSON':
                            score += 3

            # find NP dependency
            np_dep = np.root.dep_
            np_head = lemmatize(strip_punctuation(np.root.head.text))
            np_head_dep = np.root.head.dep_

            if np_dep == dep:
                score += 1
            if np_head == head:
                score += 1
            if np_head_dep == head_dep:
                score += 1

            if np.text not in question:
                score += 1

            if strip_punctuation(np.text).strip().lower() not in stop:
                score += 1

            if np.text.lower() == 'it':
                score = -1
                
            if score > max_score:
                max_score = score
                answer = np.text

    elif qtype == 'when':
        for ent in doc.ents:
            score = 0
            
            if ent.label_ == 'TIME' or ent.label_ == "DATE":
                score += 3
                
            if ent.text not in question:
                score += 1
            
            if score > max_score:
                max_score = score
                answer = ent.text

    elif qtype == 'where':
        for ent in doc.ents:
            score = 0
            
            if ent.label_ == 'GPE' or ent.label_ == "LOC":
                score += 3

            if ent.text not in question:
                score += 1
                
            if score > max_score:
                max_score = score
                answer = ent.text
            
    elif qtype in ['LANGUAGE', 'WORK_OF_ART', 'EVENT', 'NORP', 'FACILITY',
                   'GPE', 'DATE', 'TIME', 'PERCENT', 'QUANTITY', 'CARDINAL',
                   'MONEY', 'PERSON', 'ORG', 'LOC']:
        for ent in doc.ents:
            score = 0
            
            if ent.label_ == qtype:
                score += 3
            
            if ent.text not in question:
                score += 1
                
            if score > max_score:
                max_score = score
                answer = ent.text

    elif qtype == 'abrv':
        abrv = ''
        qdoc = nlp(question)
        for token in qdoc:
            text = token.text
            if len(text) >= 2 and text.isupper() and text.isalpha():
                abrv = text.lower()

        if abrv == '' and 'stand for' in question:
            tokens = question.lower().split(' ')
            abrv = tokens[tokens.index('stand')-1]

        if abrv != '':
            tokens = nltk.word_tokenize(sent)
            for (i, token) in enumerate(tokens):
                if token[0].isupper():
                    k = 1
                    phrase = token.lower()
                    initials = phrase[0]

                    while i+k < len(tokens) and tokens[i+k][0].isupper():
                        phrase = phrase + ' ' + tokens[i+k].lower()
                        initials += tokens[i+k][0].lower()
                        k += 1

                    phrase = phrase.strip()
                    if initials == abrv:
                        answer = phrase

        else:
            tokens = nltk.word_tokenize(question)
            for (i, token) in enumerate(tokens):
                if token[0].isupper():
                    k = 1
                    initials = token[0].lower()

                    while i + k < len(tokens) and tokens[i + k][0].isupper():
                        initials += tokens[i + k][0].lower()
                        k += 1

                    if len(initials) >= 2:
                        answer = initials

    elif qtype == 'adj':
        for token in doc:
            score = 0
            
            if 'advmod' in token.dep_ or 'acomp' in token.dep_:
                score += 3

            token_dep = token.dep_
            token_head = lemmatize(strip_punctuation(token.head.text))
            token_head_dep = token.head.dep_

            if token_dep == dep:
                score += 1
            if token_head == head:
                score += 1
            if token_head_dep == head_dep:
                score += 1

            if token.text not in question:
                score += 1

            if strip_punctuation(token.text).strip().lower() not in stop:
                score += 1

            if token.text.lower() == 'it':
                score = -1

            if score > max_score:
                max_score = score
                answer = token.text

    elif qtype == 'verb':
        for token in doc:
            score = 0

            if token.dep_ == 'ROOT':
                score += 1

            if lemmatize(strip_punctuation(token.text)) not in \
                    [lemmatize(strip_punctuation(s)) for s in nltk.word_tokenize(question)]:
                score += 1

            if strip_punctuation(token.text).strip().lower() not in stop:
                score += 1

            if score > max_score:
                max_score = score
                answer = token.text

    elif qtype == 'closed':
        first = closed_q_choices[0]
        second = closed_q_choices[1]

        # whether each option appears (and is negates)
        appear1 = False
        appear2 = False
        negate1 = False
        negate2 = False
        neg_count = 0
        tokens = nltk.word_tokenize(raw_docs[id])

        for (index, token) in enumerate(tokens):
            if token == 'not' or "n't" in token:
                neg_count += 1

                if index+1 < len(tokens):
                    if tokens[index+1] == first:
                        negate1 = True
                    if tokens[index+1] == second:
                        negate2 = True

            if token == first:
                appear1 = True
            if token == second:
                appear2 = True

        possible_answer = ''
        if appear1 and not appear2:
            if neg_count % 2 == 1:
                possible_answer = second
            else:
                possible_answer = first

        elif appear2 and not appear1:
            if neg_count % 2 == 0:
                possible_answer = second
            else:
                possible_answer = first

        elif appear1 and appear2:
            if negate1 and not negate2:
                possible_answer = second
            elif negate2 and not negate1:
                possible_answer = first
            else:
                possible_answer = second

        if possible_answer != '':
            score += 5
            if score > max_score:
                max_score = score
                answer = possible_answer

    elif qtype == 'why':

        possible_answer = ''
        score = 0

        if 'reason' in sent or 'because' in sent or 'due to' in sent or 'since' in sent or 'for' in sent:

            if 'because of' in sent:
                score += 3
                index = sent.index('because of')
                substr = sent[index+11:]
                span = nlp(substr)
                for chunk in span.noun_chunks:
                    possible_answer = chunk.text
                    break

            elif 'because' in sent:
                score += 3
                index = sent.index('because')
                substr = sent[index + 8:]
                possible_answer = substr

            elif 'due to' in sent:
                score += 3
                index = sent.index('due to')
                substr = sent[index+7:]
                span = nlp(substr)
                for chunk in span.noun_chunks:
                    possible_answer = chunk.text
                    break
                if possible_answer == '':
                    possible_answer = substr

            elif 'reason' in sent:
                score += 2
                index = sent.index('reason')
                substr = sent[index+7:]
                span = nlp(substr)
                for chunk in span.noun_chunks:
                    possible_answer = chunk.text
                    break
                if possible_answer == '':
                    index = substr.find('is')
                    if index != -1:
                        possible_answer = substr[index+3]
                    else:
                        index = substr.find('was')
                        if index != -1:
                            possible_answer = substr[index+4]
                        else:
                            possible_answer = sent[sent.index('reason'):]

            elif 'for' in sent:
                score += 1
                index = sent.index('for')
                substr = sent[index + 4:]
                span = nlp(substr)
                for chunk in span.noun_chunks:
                    possible_answer = chunk.text
                    break
                if possible_answer == '':
                    possible_answer = substr

            elif 'since' in sent:
                score += 1
                index = sent.index('since')
                substr = sent[index + 6:]
                possible_answer = substr

            if possible_answer != '' and score > max_score:
                answer = possible_answer
                max_score = score

    # if answer not found, find noun phrases
    if answer == '':
        for np in doc.noun_chunks:
            score = 0

            np_dep = np.root.dep_
            np_head = lemmatize(strip_punctuation(np.root.head.text))
            np_head_dep = np.root.head.dep_

            if np_dep == dep:
                score += 1
            if np_head == head:
                score += 1
            if np_head_dep == head_dep:
                score += 1

            if np.text not in question:
                score += 1

            if strip_punctuation(np.text).strip().lower() not in stop:
                score += 1

            if np.text.lower() == 'it':
                score = 0

            if score > max_score:
                max_score = score
                answer = np.text
                
    if answer != correct_answer:
        print(case_count)
        print('Question: ',question)
        print('Predicted: ',strip_punctuation(answer).strip().lower())
        print('Correct Answer: ',correct_answer)
#         print('Support sentence: ')
#         for id, _ in results:
#             sent = raw_docs[id]   
#             print(sent+'\n')
        print('\n')
    
    case_count += 1


0
Question:  A kilogram could be definined as having a Planck constant of what value?
Predicted:  several tens
Correct Answer:  6966662606895999999♠6.62606896×10−34 j⋅s


1
Question:  What is the shape of the object that establishes the base unit of the kilogram?
Predicted:  the planck constant
Correct Answer:  cylinder


2
Question:  What example is given as another paired relationship of uncertainly related to standard deviation?
Predicted:  modern terms
Correct Answer:  time vs. energy


3
Question:  What does the Planck Constant refer to?
Predicted:  r2
Correct Answer:  quantum of action


5
Question:  What scientist first studied black body radiation?
Predicted:  max planck
Correct Answer:  kirchhoff


6
Question:  Who helped to give the correct quantization rules for electrons in 1926?
Predicted:  the energy
Correct Answer:  schrödinger


8
Question:  What is required to exist in classical statistics mechanics?
Predicted:  classical statistical mechanics
Correct Answer:  h


9
Qu

63
Question:  What alloy is the base unit of the kilogram made from?
Predicted:  the planck constant
Correct Answer:  platinum–iridium


65
Question:  Who modeled the atom in 1913, challenging Rutherford's model?
Predicted:  niels bohr
Correct Answer:  niels bohr


67
Question:  What is maximized as a result of a black object absorbing all the light that hits it?
Predicted:  the object
Correct Answer:  thermal light emission


68
Question:  What is the energy of a photon?
Predicted:  possible new definitions
Correct Answer:  6981358000000000000♠3.58×10−19 j


69
Question:  What color is hotter than "red hot"?
Predicted:  the colour
Correct Answer:  white hot


70
Question:  Planck studied what problem posed originally by Kirchhoff?
Predicted:  the last years
Correct Answer:  black-body radiation


71
Question:  What is the name for the amount of energy transfered by a wave in a given time?
Predicted:  the energy
Correct Answer:  intensity


72
Question:  Einstein's paper on the photoel

132
Question:  The PV industry has seen drops in module prices since what year?
Predicted:  since 2008
Correct Answer:  2008


133
Question:  As of 2012, what accounts for about half of new nameplate electrical capacity?
Predicted:  renewable energy accounts
Correct Answer:  renewable energy


134
Question:  What is the most widely used form of renewable energy?
Predicted:  16 percent
Correct Answer:  hydroelectricity


135
Question:  In Spain, wind power accounts for what percentage of electricity generated?
Predicted:  approximately 19
Correct Answer:  9 %


136
Question:  Ban Ki-moon states that renewable energy has the ability to lift the poorest nations to new levels of prosperity?
Predicted:  united nations secretarygeneral ban kimoon
Correct Answer:  renewable energy


138
Question:  How much was the total investment in renewable energy in 2012?
Predicted:  244 billion
Correct Answer:  $ 244 billion


139
Question:  What states that EU Member States must ensure that the origin o

210
Question:  What rating did 'Temple of Doom' receive?
Predicted:  spite
Correct Answer:  pg


211
Question:  When will 'The BFG' be in most theaters?
Predicted:  may 2016
Correct Answer:  july 1 , 2016


212
Question:  Who made Spielberg a knight?
Predicted:  spielberg
Correct Answer:  jacques chirac


213
Question:  Who is naive in 'Catch Me if You Can'?
Predicted:  a career
Correct Answer:  frank


214
Question:  What was the first video game Spielberg played?
Predicted:  lucasarts
Correct Answer:  pong


216
Question:  How many Oscar nominations did 'Bridge of Spies' get?
Predicted:  one
Correct Answer:  six


217
Question:  Who wrote 'The BFG'?
Predicted:  et
Correct Answer:  roald dahl


218
Question:  Which film did Kahn first work with Spielberg on?
Predicted:  the first nonpixar film
Correct Answer:  close encounters


219
Question:  Where did Spielberg's paternal grandparents immigrate from?
Predicted:  ukraine
Correct Answer:  ukraine


220
Question:  When did the 'War Hor

289
Question:  What book was 'Munich' based on?
Predicted:  2005
Correct Answer:  vengeance


290
Question:  Who wrote the 'War of the Worlds' book?
Predicted:  spielberg
Correct Answer:  h. g. wells


291
Question:  Who wrote the 'Interstellar' script?
Predicted:  tom hanks
Correct Answer:  jonathan nolan


292
Question:  Whose son disappeared in 'Minority Report'?
Predicted:  spielberg
Correct Answer:  john anderton


293
Question:  How many Oscars did Close Encounters get nominated for, besides Best Director?
Predicted:  one
Correct Answer:  six


294
Question:  Who wrote 'The Color Purple'?
Predicted:  spielberg
Correct Answer:  alice walker


295
Question:  When was Boombots released?
Predicted:  1996
Correct Answer:  1999


296
Question:  What film festival will first air 'The BFG'?
Predicted:  competition
Correct Answer:  cannes film festival


297
Question:  When is 'Ready Player One' planned to be released?
Predicted:  early 2017
Correct Answer:  2018


298
Question:  What cha

...

2957
Question:  PAL, SECAM, and NTSC frame rates apply to what definition televisions?
Predicted:  digital or high definition broadcasts
Correct Answer:  analogue standard


2958
Question:  Which standard were ATSC and DVB based on?
Predicted:  both atsc
Correct Answer:  mpeg-2


2959
Question:  When a parameter is dropped, the remaining numeric parameter is specified first, followed by what?
Predicted:  first
Correct Answer:  the scanning system


2960
Question:  What were 1920x1080i and 1280x720p defined by?
Predicted:  some reports
Correct Answer:  us smpte standards


2962
Question:  How was the Freeview HD service rolled out in the UK?
Predicted:  the freeview hd service
Correct Answer:  region by region


2963
Question:  What is usually dropped for the commercial naming of an HDTV product?
Predicted:  a product
Correct Answer:  the frame rate


2964
Question:  What signal did the first European HDTV broadcast use?
Predicted:  these first european hdtv broadcasts
Correct Answer:  

3029
Question:  How does Oklahoma's per-capita arts spending rank?
Predicted:  is
Correct Answer:  17th


3030
Question:  Which one is more accurately placed, the TX/NM border or the OK/NM border?
Predicted:  oklahoma
Correct Answer:  oklahoma/new mexico border


3031
Question:  What is Oklahoma's rank among states producing crude oil?
Predicted:  the nations thirdlargest producer
Correct Answer:  fifth


3032
Question:  When was McVeigh executed?
Predicted:  june 11 2001
Correct Answer:  june 11 , 2001


3033
Question:  What is the third-most-prevalent language in Oklahoma?
Predicted:  arabic
Correct Answer:  cherokee


3034
Question:  What is the busiest highway in Oklahoma?
Predicted:  2008
Correct Answer:  interstate 44


3036
Question:  Where has the Medieval Fair of Norman been held since 2003?
Predicted:  oklahoma
Correct Answer:  reaves park


3037
Question:  What are Oklahoma's Native American ballerinas called?
Predicted:  the state
Correct Answer:  the five moons


3038
Ques

3103
Question:  Who commands the Oklahoma National Guard?
Predicted:  the principal head
Correct Answer:  the governor


3104
Question:  What was Spiro, Oklahoma named for?
Predicted:  the westernmost center
Correct Answer:  spiro mounds


3105
Question:  What percent of Oklahomans speak only English at home, as of 2000?
Predicted:  english
Correct Answer:  92.6 %


3106
Question:  How many square miles is Oklahoma?
Predicted:  181035
Correct Answer:  69,898


3107
Question:  How many ports does Oklahoma have?
Predicted:  two
Correct Answer:  two


3108
Question:  Where is 'Discoveryland!'?
Predicted:  oklahoma
Correct Answer:  sand springs


3109
Question:  What parts of Oklahoma rarely see below-zero temperatures?
Predicted:  the state
Correct Answer:  south-central and southeastern


3110
Question:  How many tornadoes hit Oklahoma each year?
Predicted:  about one
Correct Answer:  average 62


3112
Question:  What does Oklahoma spend per student in public schools as of 2008?
Predicte

3188
Question:  An activist in the Arab spring said they were using youtube to do what?
Predicted:  quoted
Correct Answer:  tell the world


3189
Question:  What was the highest earning youtube partner video producer?
Predicted:  preroll advertisements
Correct Answer:  $ 12 million


3190
Question:  What was the original title of youtube red?
Predicted:  the announcement
Correct Answer:  music key


3191
Question:  How much content on youtube comes from outside the US?
Predicted:  2014
Correct Answer:  around three quarters


3192
Question:  As of 2010 what is the maximum length of a video a user can upload given the proper authority?
Predicted:  youtube
Correct Answer:  unlimited


3193
Question:  How many independent labels did the Merlin Network represent?
Predicted:  over 20000
Correct Answer:  20,000


3194
Question:  How many copyright infringements must an account have before the account is deleted?
Predicted:  three
Correct Answer:  three


3195
Question:  What type of videos d

ValueError: list.remove(x): x not in list